In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gmr import GMM

from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import f1_score, accuracy_score

from moebius.model.gmm.evol.optimizer import GMMOptimizer
from moebius.model.gmm.evol.parameters import EvolutionaryConfiguration
from moebius.model.gmm.evol.graph.graph import ProbabilisticGraphShuffler

from moebius.model.gmm.evol.graph.graph import GeneratorNode
from moebius.model.gmm.evol.operators.mutation import mutation, \
    Mutation, \
    ByNameNodeSelector, \
    RandomValueNodeMutator, \
    RandomDeltaNodeMutator, \
    ClapNodeDecoratedMutator, \
    RandomIndexRandomDeltaNodeMutator, \
    RandomValuesSumTo1NodeMutator, \
    DirichletValuesSumTo1NodeMutator

from moebius.model.gmm.evol.operators.crossover import crossover, \
    Crossover, \
    ExchangeFieldNodesCrossover, \
    RandomIndexExchangeFieldNodesCrossover

from moebius.model.gmm.evol.graph.initializer import GMMParametersInitializer, \
    RandomWeightsInitializer, \
    RandomMeansInitializer, \
    RandomCovariancesInitializer, \
    ConstantMeansInitializer, \
    ConstantCovariancesInitializer

from moebius.model.gmm.evol.graph.auto_initializer import AutoMeansInitializer

from moebius.model.gmm.bn import ContinuousBNSerializer, JsonDictSerializer, FilesystemStorage

from moebius.model.gmm.evol.bn import build_bn
from moebius.util import make_diagonal, print_gmm_params_fancy
from moebius import olr, olr_universal

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def explode_cluster(data, target, clusters):
    indices_0_1 = [idx for idx, x in enumerate(target) if x in clusters]
    
    return (
        [list(data[idx]) for idx in indices_0_1],
        [target[idx].decode('UTF-8') if isinstance(target[idx], np.bytes_) else target[idx] for idx in indices_0_1]
    )

In [3]:
data = load_iris()
X = data['data']
y = data['target']

In [4]:
X_1_2, y_1_2 = explode_cluster(X, y, (1, 2))

In [5]:
gmm = GMM(n_components=2)
gmm.from_samples(np.array(X_1_2))

w = gmm.priors.tolist()
means = gmm.means.tolist()
cov = make_diagonal(gmm.covariances.tolist())

print_gmm_params_fancy(w, means, cov, decimal_places=3)

 ---  weights   --- 
[0.551 0.449]
 ---   means    --- 
[[6.545 2.949 5.48  1.985]
 [5.915 2.778 4.202 1.297]]
 --- covariance --- 
[[[0.387 0.092 0.303 0.062]
  [0.092 0.11  0.084 0.056]
  [0.303 0.084 0.328 0.075]
  [0.062 0.056 0.075 0.086]]

 [[0.275 0.097 0.185 0.054]
  [0.097 0.093 0.091 0.043]
  [0.185 0.091 0.201 0.061]
  [0.054 0.043 0.061 0.032]]]
 ---    OLR     --- 
0.3201848698063574


In [6]:
pd.DataFrame(data=X_1_2).describe()

,0,1,2,3
count,100.000000,100.000000,100.000000,100.000000
mean,6.262000,2.872000,4.906000,1.676000
std,0.662834,0.332751,0.825578,0.424769
min,4.900000,2.000000,3.000000,1.000000
25%,5.800000,2.700000,4.375000,1.300000
50%,6.300000,2.900000,4.900000,1.600000
75%,6.700000,3.025000,5.525000,2.000000
max,7.900000,3.800000,6.900000,2.500000


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_1_2, y_1_2, test_size=0.25, random_state=0)

In [8]:
model = LogisticRegression()
model.fit(X_train, y_train)

accuracy = accuracy_score(y_test, model.predict(X_test))
f1 = f1_score(y_test, model.predict(X_test), pos_label=2)
accuracy, f1

(0.92, 0.923076923076923)

In [9]:
auto = AutoMeansInitializer.from_data(X_1_2, 2)
auto.create_initial_means('Comp_3', 2)

[[1.2969685652784884], [1.9846085276174643]]

In [10]:
optimizer = GMMOptimizer(
    EvolutionaryConfiguration()
        .set_mutation_types([
            mutation(RandomValueNodeMutator('var', [
                [
                    [0.0, 0.25],
                ],
                [
                    [0.0, 0.25],
                ],
            ])),
            mutation(RandomValuesSumTo1NodeMutator('w')),
        ])
        .set_crossover_types([
            crossover(ExchangeFieldNodesCrossover('var')),
            crossover(RandomIndexExchangeFieldNodesCrossover('var'))
        ]),
    ProbabilisticGraphShuffler(0.5),
    GMMParametersInitializer(
        RandomWeightsInitializer(),
#         ConstantMeansInitializer({
#             'Comp_0': [
#                 [6.545],
#                 [5.915]
#             ],
#             'Comp_1': [
#                 [2.949],
#                 [2.778]
#             ],
#             'Comp_2': [
#                 [5.48],
#                 [4.202]
#             ],
#             'Comp_3': [
#                 [1.985],
#                 [1.297]
#             ]
#         }),
        AutoMeansInitializer.from_data(X_1_2, 2),
        RandomCovariancesInitializer(0, 2)
    )
)

In [11]:
optimization_result = optimizer.optimize(n_dim=4, n_comp=2, target_olr=0.3)

Generations:   0%|          | 1/500 [00:00<?, ?gen/s]

2024-04-14 12:57:56,667 - MultiprocessingDispatcher - Number of used CPU's: 6


 52%|█████▏    | 2585/5000 [00:00<00:00, 12931.01it/s]

2024-04-14 12:57:58,744 - MultiprocessingDispatcher - 1 individuals out of 1 in previous population were evaluated successfully.
2024-04-14 12:57:58,747 - EvoGraphOptimizer - Generation num: 1 size: 1
2024-04-14 12:57:58,747 - EvoGraphOptimizer - Best individuals: HallOfFame archive fitness (1): ['<custom=0.354>']


100%|██████████| 5000/5000 [00:00<00:00, 11018.83it/s]


...

Error message
[Errno 17] File exists: '/root/BAMT/Nodes_data/6870'


 96%|█████████▌| 4783/5000 [00:00<00:00, 11958.18it/s]

2024-04-14 12:59:41,211 - MultiprocessingDispatcher - 45 individuals out of 54 in previous population were evaluated successfully.
2024-04-14 12:59:41,214 - EvoGraphOptimizer - Generation num: 24 size: 55
2024-04-14 12:59:41,215 - EvoGraphOptimizer - Best individuals: HallOfFame archive fitness (1): ['<custom=0.000>']
2024-04-14 12:59:41,215 - EvoGraphOptimizer - no improvements for 5 iterations
2024-04-14 12:59:41,216 - EvoGraphOptimizer - spent time: 1.7 min
2024-04-14 12:59:41,216 - GroupedCondition - Optimisation finished: Early stopping iterations criteria was satisfied


Generations:   0%|          | 1/500 [01:44<?, ?gen/s]

2024-04-14 12:59:41,217 - OptimisationTimer - Composition time: 1.743 min
2024-04-14 12:59:41,217 - EvoGraphOptimizer - Generation num: 25 size: 1
2024-04-14 12:59:41,218 - EvoGraphOptimizer - Best individuals: HallOfFame archive fitness (1): ['<custom=0.000>']
2024-04-14 12:59:41,219 - EvoGraphOptimizer - no improvements for 6 iterations
2024-04-14 12:59:41,219 - EvoGraphOptimizer - spent time: 1.7 min


In [12]:
optimized_graph = optimization_result[0]
net = build_bn(optimized_graph)

In [13]:
data = net.sample(200)

100%|██████████| 200/200 [00:00<00:00, 807.84it/s]


In [14]:
data.describe()

,Comp_0,Comp_1,Comp_2,Comp_3
count,200.000000,200.000000,200.000000,200.000000
mean,6.238868,2.906282,5.141847,2.058017
std,0.487391,0.208641,0.647394,1.395904
min,4.937032,2.146774,3.521673,-1.224117
25%,5.883381,2.787700,4.690504,1.109719
50%,6.285163,2.918602,5.374628,1.964388
75%,6.557211,3.039409,5.620151,2.965573
max,7.342145,3.573231,6.161262,5.593079


In [15]:
gmm = GMM(n_components=2)
gmm.from_samples(data.to_numpy())

w = gmm.priors.tolist()
means = gmm.means.tolist()
cov = make_diagonal(gmm.covariances.tolist())

# print_gmm_params_fancy(w, means, cov, decimal_places=3)

olr_universal(w, means, cov)

0.29827803905987227

In [16]:
net

In [16]:
serialized = ContinuousBNSerializer().serialize(net)
j = JsonDictSerializer().serialize(serialized)
FilesystemStorage().write(j, './bns_iris/0.3_1.json')

In [17]:
gmm = GaussianMixture(n_components = 2) 
gmm.fit(data)
        
sample = gmm.sample(200)
data = sample[0]
labels = [2 if each == 1 else 1 for each in sample[1]]

accuracy = accuracy_score(labels, model.predict(data))
f1 = f1_score(labels, model.predict(data))

accuracy, f1

(0.93, 0.933477021642587)